1. Setup

In [ ]:
! pip install pinecone-client


In [ ]:
! pip install langchain[extras]


In [12]:
import pinecone

print("Pinecone instalado y funcionando correctamente.")


Pinecone instalado y funcionando correctamente.


In [13]:
# Importar bibliotecas
import json

# Cargar el archivo faq.json
with open("textos/faq.json", "r", encoding="utf-8") as file:
    faq_data = json.load(file)

# Verificar que faq.json esté cargado correctamente
print("Preguntas frecuentes cargadas:", len(faq_data))


Preguntas frecuentes cargadas: 9


2. Define Function for answering quesitons

In [14]:
def chatbot(query):
    # Buscar preguntas similares en faq_data
    for entry in faq_data:
        if query.lower() in entry["question"].lower():
            return entry["answer"]
    return "Lo siento, no tengo información para esa pregunta. ¿Puedes reformularla?"


3. verify json

In [ ]:

# Cargar el JSON
with open("textos/faq.json", "r", encoding="utf-8") as file:
    faq_data = json.load(file)

# Verificar las primeras entradas
print("Total de entradas:", len(faq_data))
print("Primeras 3 preguntas:")
for entry in faq_data[:3]:  # Muestra las primeras 3 preguntas
    print("Pregunta:", entry["question"])
    print("Respuesta:", entry["answer"])
    print()


**Embeddings Generation and Pincode**

1. SETUP

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
import json
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENV")  # Región de Pinecone


In [17]:
# Verificar que las claves están cargadas
print("PINECONE_API_KEY:", pinecone_api_key)
print("PINECONE_ENV:", pinecone_env)


PINECONE_API_KEY: pcsk_pb1LW_2rvzUder7H9V8A4p7ehWRv5DjJttpATaaBxKgSZEkJEp6UtGwT4g8CzKzZ3xLWg
PINECONE_ENV: eu-west-1


2. Setup of Q&A con Pincone

In [18]:
from pinecone import Pinecone

# Crear una instancia de Pinecone con tu API Key
pc = Pinecone(api_key=pinecone_api_key)

# Listar índices disponibles
existing_indexes = pc.list_indexes()
print("Índices existentes:", [i["name"] for i in existing_indexes])


Índices existentes: ['alejandra-project', 'alejandra-project2', 'about-conditions-index', 'alejandra-combined']


In [19]:
from pinecone import Pinecone, ServerlessSpec



In [20]:
# Crear una instancia de Pinecone
pc = Pinecone(api_key=pinecone_api_key)

# Especificar la región y proveedor (serverless)
spec = ServerlessSpec(cloud="aws", region=pinecone_env)

# Crear un índice para el proyecto de Alejandra
index_name = "alejandra-project2"
if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # Dimensiones para text-embedding-ada-002
        metric="cosine",  # Métrica de similitud                                      other options:   dotproduct
        spec=spec
    )
    print(f"Índice '{index_name}' creado con éxito.")
else:
    print(f"El índice '{index_name}' ya existe.")

# Conectar al índice
index = pc.Index(index_name)
print(f"Índice '{index_name}' conectado.")

El índice 'alejandra-project2' ya existe.
Índice 'alejandra-project2' conectado.


**USE DATASET** 

In [ ]:
! pip install tqdm langchain pandas

In [2]:
!pip install pandas


------mepezar desde aqui -----

In [ ]:
import pandas as pd

# Cargar el archivo CSV con pandas
try:
    faq_data = pd.read_csv("faq_dataset.csv", encoding="utf-8")
    print(faq_data.head())  # Muestra las primeras filas para verificar
except Exception as e:
    print("Error al cargar el archivo CSV:", e)



In [21]:
faq_data = pd.read_csv("faq_dataset.csv", encoding="utf-8-sig")


In [ ]:
# Cargar el archivo CSV
faq_data = pd.read_csv("faq_dataset.csv", encoding="utf-8")

# Iterar sobre preguntas y respuestas
for _, row in faq_data.iterrows():
    question = row["question"]
    answer = row["answer"]
    print(f"Pregunta: {question}, Respuesta: {answer}")


**EMBEDDINGS**

In [23]:
from tqdm.auto import tqdm
from langchain.embeddings.openai import OpenAIEmbeddings

# Configurar OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Subir los embeddings al índice en lotes
batch_size = 100
questions = faq_data["question"].tolist()
answers = faq_data["answer"].tolist()

for i in tqdm(range(0, len(questions), batch_size)):
    # Obtener el lote actual
    i_end = min(len(questions), i + batch_size)
    batch_questions = questions[i:i_end]
    batch_answers = answers[i:i_end]

    # Generar embeddings
    batch_embeddings = embeddings.embed_documents(batch_questions + batch_answers)
   

    # Generar IDs únicos y metadatos
    ids = [f"q-{j}" for j in range(i, i_end)]
    metadatas = [{"question": q, "answer": a} for q, a in zip(batch_questions, batch_answers)]

    # Subir a Pinecone
    index.upsert(vectors=zip(ids, batch_embeddings, metadatas))

print("Embeddings generados y subidos al índice.")


  0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generados y subidos al índice.


In [24]:
# Verificar el estado del índice
stats = index.describe_index_stats()
print(f"Estadísticas del índice: {stats}")


Estadísticas del índice: {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}


In [ ]:
# Realizar una consulta de prueba
query = "¿Qué pasa en una terapia psicológica?"
query_embedding = embeddings.embed_query(query)

# Buscar en Pinecone usando argumentos con nombre
results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

# Mostrar los resultados
for match in results["matches"]:
    print(f"Pregunta: {match['metadata']['question']}")
    print(f"Respuesta: {match['metadata']['answer']}")
    print(f"Similitud: {match['score']}")
